In [2]:
import underworld as uw
from underworld import function as fn
from underworld import visualisation as glucifer
import numpy as np
import math
import warnings
import os,csv
warnings.filterwarnings("ignore")

from underworld.scaling import units as u
from underworld.scaling import non_dimensionalise as nd

outputPath = os.path.join(os.path.abspath("."),"Reynolds_Tube_FixBall2/")
if uw.mpi.rank == 0:
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)
uw.mpi.barrier()

scaling_coefficients = uw.scaling.get_coefficients()

tempMin = 273.*u.degK 
tempMax = (1400.+ 273.)*u.degK
bodyforce = 3300 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2
velocity = 1.0 *u.meter/u.second

KL = 0.1*u.meter
Kt = KL/velocity
KT = tempMax 
KM = bodyforce * KL**2 * Kt**2
K  = 1.*u.mole

scaling_coefficients["[length]"] = KL
scaling_coefficients["[time]"] = Kt
scaling_coefficients["[mass]"]= KM
scaling_coefficients["[temperature]"] = KT
scaling_coefficients["[substance]"] = K


gravity = 0*nd(9.81 * u.meter / u.second**2)
R = nd(8.3144621 * u.joule / u.mole / u.degK)

time_factor=nd(1*u.second)
vis_factor = nd(1*u.pascal*u.second)
vel_factor = nd(1*u.meter/u.second) 
strainRate_factor = nd(1/u.second)
length_factor = nd(1*u.centimeter)
stress_factor = nd(1*u.pascal)

Setup parameters
-----

Set simulation parameters for the test and position of the cylinder.

In [3]:
# Set the resolution.
resx = 64
resy = 128
# Set size and position of dense sphere.
sphereRadius = nd(2.*1.27/2*u.centimeter)
sphereCentre = (0.,nd(80.*u.centimeter))
minX = nd(-12*u.centimeter)
maxX = nd(12*u.centimeter)
minY = 0.
maxY = nd(120.*u.centimeter)

Vy  =  -nd(1.0*u.meter/u.second)

In [6]:
vis = nd(0.031*u.pascal*u.second)
rho = nd(876.*u.kilogram/u.meter**3)
D = nd(1.27*u.centimeter)
v = nd(2*u.meter/u.second)
Re = rho*v*D/vis

### Check the desinged Reynolds nunber

In [9]:
print(Re)

717.7548387096776


Create mesh and finite element variables
------

In [10]:
mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1/dQ0"), 
                                 elementRes  = (resx, resy), 
                                 minCoord    = (minX, minY), 
                                 maxCoord    = (maxX, maxY),
                                 periodic    =  [False, False] )

velocityField    = mesh.add_variable(         nodeDofCount=2 )
pressureField    = mesh.subMesh.add_variable( nodeDofCount=1 )
stressField      = mesh.add_variable(         nodeDofCount=2 )
step = 0 
if uw.mpi.rank == 0:

    dicMesh = { 'elements' : mesh.elementRes, 
                'minCoord' : mesh.minCoord,
                'maxCoord' : mesh.maxCoord}

    fo = open(outputPath+"dicMesh"+str(step).zfill(4),'w')
    fo.write(str(dicMesh))
    fo.close()  

uw.mpi.barrier()
    

	Global element size: 64x128
	Local offset of rank 0: 0x0
	Local range of rank 0: 64x128


Set initial conditions and boundary conditions
----------

**Initial and boundary conditions**

Initialise the velocity and pressure fields to zero.

In [12]:
velocityField.data[:] = [0.,0.]
pressureField.data[:] = 0.

**Conditions on the boundaries**

Construct sets for the both horizontal and vertical walls to define conditons for underworld solvers.

In [ ]:
iWalls = mesh.specialSets["MinI_VertexSet"] + mesh.specialSets["MaxI_VertexSet"]
jWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
top = mesh.specialSets["MaxJ_VertexSet"]
base = mesh.specialSets["MinJ_VertexSet"]

Ball = mesh.specialSets['Empty']
#make the ball/cylinder fixed
for index, coord in enumerate(mesh.data):
    x = coord[0]
    y = coord[1]
    sphere =  (x - sphereCentre[0])**2+(y - sphereCentre[1])**2

    if sphere < sphereRadius**2:
        Ball+=index
        
for index in mesh.specialSets["MaxJ_VertexSet"]:
    velocityField.data[index] = Vy

### Boundary condition for the example of a falling cylinder 

In [ ]:
# freeslipBC = uw.conditions.DirichletCondition( variable      = velocityField, 
#                                                indexSetsPerDof = (iWalls,iWalls+jWalls) )

### Boundary condition for the example of  a fixed cylinder

In [13]:
freeslipBC = uw.conditions.DirichletCondition( variable      = velocityField, 
                                               indexSetsPerDof = (iWalls,Ball+jWalls) )

Create a particle swarm
------

Swarms refer to (large) groups of particles which can advect with the fluid flow. These can be used to determine 'materials' as they can carry local information such as the fluid density and viscosity.

**Setup a swarm**

To set up a swarm of particles the following steps are needed:
1. Initialise and name a swarm, here called ``swarm``.
2. Define data variable (``materialIndex``) to store an index that will state what material a given particle is.
3. Populate the swarm over the whole domain using the layout command, here this is used to allocate 20 particles in each element.

In [14]:
# Create the swarm and an advector associated with it
swarm = uw.swarm.Swarm( mesh=mesh,particleEscape=True )
advector = uw.systems.SwarmAdvector( swarm=swarm, velocityField=velocityField, order=2 )

# Add a data variable which will store an index to determine material.
materialIndex = swarm.add_variable( dataType="int", count=1 )
previousVm =  swarm.add_variable( dataType="double", count=2 )
velSwarm =  swarm.add_variable( dataType="double", count=2 )
densitySwarm =  swarm.add_variable( dataType="double", count=1 )

# Create a layout object that will populate the swarm across the whole domain.
swarmLayout = uw.swarm.layouts.PerCellGaussLayout( swarm=swarm, gaussPointCount=5 )
pop_control = uw.swarm.PopulationControl(swarm,aggressive=True,particlesPerCell=20)

# Go ahead and populate the swarm.
swarm.populate_using_layout( layout=swarmLayout )

**Define a shape**
 1. Define an underworld `function` that descripes the geometry of a shape (a sphere), called `fn_sphere`.
 2. Set up a `fn.branching.conditional` to run `fn_sphere` and return a given index - either `materialLightIndex` or `materialHeavyIndex`.
 3. Execute the above underworld functions on the swarm we created and save the result on the `materialIndex` swarm variable.
 

In [15]:
# create a function for a sphere. returns `True` if query is inside sphere, `False` otherwise.
coord = fn.input() - sphereCentre
fn_sphere = fn.math.dot( coord, coord ) < sphereRadius**2

# define some names for our index 
materialLightIndex = 0
materialHeavyIndex = 1

# set up the condition for being in a sphere. If not in sphere then will return light index.
conditions = [ ( fn_sphere , materialHeavyIndex), 
               ( True      , materialLightIndex) ]

# Execute the branching conditional function, evaluating at the location of each particle in the swarm.
# The results are copied into the materialIndex swarm variable.
materialIndex.data[:] = fn.branching.conditional( conditions ).evaluate(swarm)

**Branching conditional function**

For more information on the `fn.branching.conditional` see the Functions user guide [here](../user_guide/05_Functions.ipynb)

**Define minimum y coordinate function**

 1. Define a new swarm called `tracerSwarm`, with one particle at the base of the sphere. (This swarm behaves as a passive tracer swarm).
 2. Define a function that finds the minimum y coordinate value of the `tracerSwarm` in a parallel safe way.

In [16]:
# build a tracer swarm with one particle
tracerSwarm = uw.swarm.Swarm(mesh,particleEscape=True)
advector_tracer = uw.systems.SwarmAdvector( swarm=tracerSwarm, velocityField=velocityField, order=2 )

# build a numpy array with one particle, specifying it's exact location
x_pos = sphereCentre[0]
y_pos = sphereCentre[1]#-sphereRadius
coord_array = np.array(object=(x_pos,y_pos),ndmin=2)
tracerSwarm.add_particles_with_coordinates(coord_array)

# define a y coordinate `min_max` function
fn_ycoord = fn.view.min_max( fn.coord()[1] )

def GetSwarmYMin():
    fn_ycoord.reset()
    fn_ycoord.evaluate(tracerSwarm),velocityField.evaluate(tracerSwarm.particleCoordinates.data)
    return fn_ycoord.min_global(),velocityField[1].evaluate(tracerSwarm.particleCoordinates.data)

In [17]:
tracerSwarm.data[:]

array([[ 0.,  8.]])

### Design band swarm to visualize flow patterns

In [19]:
bandSwarm = uw.swarm.Swarm(mesh,particleEscape=True)
advector_band = uw.systems.SwarmAdvector( swarm=bandSwarm, velocityField=velocityField, order=2 )
material_band = bandSwarm.add_variable( dataType="int", count=1 )
#swarmLayout_band = uw.swarm.layouts.PerCellGaussLayout( swarm=bandSwarm, gaussPointCount=1 )
swarmLayout_band = uw.swarm.layouts.PerCellSpaceFillerLayout( swarm=bandSwarm, particlesPerCell=8 )
pop_control_band = uw.swarm.PopulationControl(bandSwarm,aggressive=True,particlesPerCell=8)

bandSwarm.populate_using_layout( layout=swarmLayout_band )

y  = fn.input()[1]
x  = fn.input()[0]
nband = 19
bW = (maxX-minX)/nband/2
x_c = np.linspace(minX,maxX,nband)
# conditions_band = [ (fn_sphere, 3),
#                     ( (y > sphereCentre[1]+sphereRadius)&(fn.math.abs(x-x_c[1])<bW) , 1),
#                     ( (y > sphereCentre[1]+sphereRadius)&(fn.math.abs(x-x_c[3])<bW) , 2),
#                     ( (y > sphereCentre[1]+sphereRadius)&(fn.math.abs(x-x_c[5])<bW) , 3),
#                     ( (y > sphereCentre[1]+sphereRadius)&(fn.math.abs(x-x_c[7])<bW) , 1),
#                     ( (y > sphereCentre[1]+sphereRadius)&(fn.math.abs(x-x_c[9])<bW) , 2),
#                     ( (y > sphereCentre[1]+sphereRadius)&(fn.math.abs(x-x_c[10])<bW) ,3),
#                     ( (y > sphereCentre[1]+sphereRadius)&(fn.math.abs(x-x_c[13])<bW) ,1),
#                     ( (y > sphereCentre[1]+sphereRadius)&(fn.math.abs(x-x_c[15])<bW) ,2),
#                     ( (y > sphereCentre[1]+sphereRadius)&(fn.math.abs(x-x_c[17])<bW) ,3),
#                      (True,      0) ]

conditions_band = [ (fn_sphere, 4),
                    ((fn.math.abs(x-x_c[1])<bW) , 1),
                    ((fn.math.abs(x-x_c[3])<bW) , 2),
                    ((fn.math.abs(x-x_c[5])<bW) , 3),
                    ((fn.math.abs(x-x_c[7])<bW) , 1),
                    ((fn.math.abs(x-x_c[9])<bW) , 2),
                    ((fn.math.abs(x-x_c[11])<bW) , 3),
                    ((fn.math.abs(x-x_c[13])<bW) , 1),
                    ((fn.math.abs(x-x_c[15])<bW) , 2),
                    ((fn.math.abs(x-x_c[17])<bW) , 3),
                     (True,      0) ]

material_band.data[:] =  fn.branching.conditional( conditions_band ).evaluate(bandSwarm)

** Test minimum y coordinate function**

In [20]:
outPutTrack = GetSwarmYMin()
print('Minimum y value for sinker = {0:.3e},velocity = {0:.3e})'.format(outPutTrack[0],outPutTrack[1]))

Minimum y value for sinker = 8.000e+00,velocity = 8.000e+00)


**Plot the particles by material**

Plot the initial positions of all swarm particles coloured by their material indices.

In [33]:
fig1 = glucifer.Figure( figsize=(600,1400) )
fig1.Points(swarm, material, colourBar=True, pointSize=1.3)
fig1.VectorArrows(mesh, velocityField,scale=1e3)
fig1.show()

NameError: name 'material' is not defined

Set up material parameters and functions
-----

Here the functions for density and viscosity are set using the ``map`` function. This function evaluates a key function (here the material index), and the result (i.e. the key) is used to determine which function to evaluate to obtain the actual result (such as the particle density). 


In [22]:
# Set constants for the viscosity and density of the sinker.
viscSphere = nd(1e6*u.pascal*u.second)
densitySphere = nd(7780. * u.kilogram / u.metre**3 )

# Here we set a viscosity value of '1.' for both materials  nd(0.31e-1*u.pascal*u.second)
mappingDictViscosity = { materialLightIndex:nd(0.31e-1*u.pascal*u.second), materialHeavyIndex:viscSphere }
# Create the viscosity map function.
viscosityMapFn = fn.branching.map( fn_key=materialIndex, mapping=mappingDictViscosity )
# Here we set a density of '0.' for the lightMaterial, and '1.' for the heavymaterial.
mappingDictDensity = { materialLightIndex:nd(876. * u.kilogram / u.metre**3 ), materialHeavyIndex:densitySphere }
# Create the density map function.
densityFn = fn.branching.map( fn_key=materialIndex, mapping=mappingDictDensity )

# And the final buoyancy force function.
z_hat = ( 0.0, -0.0 )
buoyancyFn = densityFn * z_hat * gravity




In [24]:
previousVm.data[:] = 0.
dt_e = fn.misc.constant(nd(1e-1*u.second))

LHS_fn = densityFn/dt_e 
RHS_fn = densityFn*previousVm/dt_e

In [25]:
stokes0 = uw.systems.Stokes(    velocityField = velocityField, 
                               pressureField = pressureField,
                               #voronoi_swarm = swarm, 
                               conditions    = [freeslipBC,],
                               fn_viscosity  = viscosityMapFn, 
                               #fn_bodyforce  = buoyancyFn)
                               #fn_stresshistory = inertiaFn,
                               fn_bodyforce  = buoyancyFn+RHS_fn)
                               #fn_one_on_lambda  = oneonlambdaFn)




massMatrixTerm = uw.systems.sle.MatrixAssemblyTerm_NA__NB__Fn(
                        assembledObject  = stokes0._kmatrix,
                        integrationSwarm = stokes0._constitMatTerm._integrationSwarm,
                        fn   = LHS_fn,
                        mesh = mesh)

solver0 = uw.systems.Solver( stokes0 )
solver0.set_inner_method('mumps') 
solver0.set_penalty(1e3)

In [26]:
top = mesh.specialSets["MaxJ_VertexSet"]
surfaceArea = uw.utils.Integral(fn=1.0,mesh=mesh, integrationType='surface', surfaceIndexSet=top)
surfacePressureIntegral = uw.utils.Integral(fn=pressureField, mesh=mesh, integrationType='surface', surfaceIndexSet=top)

# a callback function to calibrate the pressure - will pass to solver later
def pressure_calibrate():
    (area,) = surfaceArea.evaluate()
    (p0,) = surfacePressureIntegral.evaluate()
    offset = p0/area
    #print("Zeroing pressure using mean upper surface pressure {}".format( offset ))
    pressureField.data[:] -= offset
    #velSwarm.data[:] = velocityField.evaluate(swarm)


Analysis tools
-----

**RMS velocity**

Set up integrals used to calculate the RMS velocity.

In [27]:
vdotv = fn.math.dot( velocityField, velocityField )

Main simulation loop
-----

The main time stepping loop begins here. Before this the time and timestep are initialised to zero and the output statistics arrays are set up. Also the frequency of outputting basic statistics to the screen is set in steps_output.

Note that there are two ``advector.integrate`` steps, one for each swarm, that need to be done each time step.

In [28]:
# define an update function
def update():
    # Retrieve the maximum possible timestep for the advection system.
    
    dt1 = advector.get_max_dt()
    dt2 = dt_e.value
    dt = min(dt1,dt2)
    
    dt_e.value = dt
    print ("dt_adv=",dt1/nd(1.*u.second),"dt_e=",dt_e.value/nd(1.*u.second),"dt=",dt/nd(1.*u.second))        

    previousVm.data[:] =  velocityField.evaluate(swarm) #( phi*previousVm_data[:] + ( 1.-phi )*previousVm.data[:] )
    
    
    #densitySwarm = densityFn.evaluate(swarm)
    
    # Advect using this timestep size.    
    advector.integrate(dt)
    advector_tracer.integrate(dt)
    advector_band.integrate(dt)
    
    pop_control.repopulate()
    pop_control_band.repopulate()
    
    return time+dt, step+1

In [29]:
# Stepping. Initialise time and timestep.
time = 0.
step = 0
nsteps = 1

tSinker = np.zeros(nsteps)
ySinker = np.zeros([nsteps,3])
steps_output = 20

if step == 0:
    title = ['step','time','Y','Vy']
    with open(outputPath+'Sample.csv', 'w') as f:
        csv_write = csv.writer(f)
        csv_write.writerow(title)
        
fn_Vy = fn.view.min_max(velocityField[1])  

# Perform 10 steps
while step<nsteps:

    
    solver0.solve()
    if step % steps_output == 0 or step == nsteps-1:
        mesh.save(outputPath+"mesh"+str(step).zfill(4)) 
        swarm.save(outputPath+"swarm"+str(step).zfill(4))
        materialIndex.save(outputPath+"materialVariable"+str(step).zfill(4))
        velocityField.save(outputPath+"velocityField"+str(step).zfill(4))
        bandSwarm.save(outputPath+"bandSwarm"+str(step).zfill(4))
        material_band.save(outputPath+"materialBand"+str(step).zfill(4))
        
    outPutTrack = GetSwarmYMin()
    # Calculate the RMS velocity
    vrms = math.sqrt( mesh.integrate(vdotv)[0] / mesh.integrate(1.)[0] )
    if uw.mpi.rank==0:
        ySinker[step,0] = outPutTrack[0]
        ySinker[step,1] = outPutTrack[1]
        tSinker[step] = time
        print('step = {0:6d}; time = {1:.3e}; v_rms = {2:.3e}; height = {3:.5e}'
              .format(step,time/time_factor,vrms/vel_factor,outPutTrack[0]/length_factor))
        print  ("Vy-max=",np.max(-velocityField.data[:,1]/nd(1.*u.meter/u.second)))
        fn_ycoord.evaluate(tracerSwarm)
    fn_Vy.evaluate(tracerSwarm)
    velocityField.evaluate(tracerSwarm.particleCoordinates.data)
    
    ySinker[step,0] = fn_ycoord.min_global()
    ySinker[step,1] = fn_Vy.min_global()
    ySinker[step,2] = time
    tSinker[step] = time       
    
    if uw.mpi.rank == 0:
  
        dicMesh = { 'elements' : mesh.elementRes, 
                    'minCoord' : mesh.minCoord,
                    'maxCoord' : mesh.maxCoord}

        fo = open(outputPath+"dicMesh"+str(step).zfill(4),'w')
        fo.write(str(dicMesh))
        fo.close()  
        
    uw.mpi.barrier()
    
    if uw.mpi.rank== 0:
        SP_output = [step,time, ySinker[step,0],ySinker[step,1]]
        with open(outputPath+'Sample.csv', 'a') as f:
            csv_write = csv.writer(f)  
            csv_write.writerow(SP_output)
    uw.mpi.barrier()        
    
    # update
    time, step = update()

Linear solver (OO4DX61N__system-execute) 

BSSCR -- Block Stokes Schur Compliment Reduction Solver 


-----  K2_GMG  ------

AUGMENTED LAGRANGIAN K2 METHOD - Penalty = 1000.000000


	* K+p*K2 in time: 0.011886 seconds

  Setting schur_pc to "gkgdiag" 


SCR Solver Summary:

  RHS V Solve:            = 0.02989 secs / 1 its
  Pressure Solve:         = 206.2 secs / 10000 its
  Final V Solve:          = 0.02037 secs / 1 its

  Total BSSCR Linear solve time: 210.372666 seconds

Linear solver (OO4DX61N__system-execute), solution time 2.103912e+02 (secs)
step =      0; time = 0.000e+00; v_rms = 1.196e+05; height = 8.00000e+01
Vy-max= 628270.848263
dt_adv= 2.9837138618742044e-09 dt_e= 2.9837138618742044e-09 dt= 2.9837138618742044e-09
Time Integration
	2nd order:                 EX9JD1SA__integrand -    0.1462 [min] /    0.1462 [max] (secs)
Time Integration - 0.146288 [min] / 0.146288 [max] (secs)
Time Integration
	2nd order:                 90CDJ7S1__integrand -    0.0000 [min] /    0.0000 [ma